In [8]:
from gmail import Gmail
from datetime import datetime
from setfit import SetFitModel
from bs4 import BeautifulSoup
import os
import pandas as pd
from flan import JobTitleCompanyNameExtractor




In [4]:
email_clasification_model = SetFitModel.from_pretrained("./../models/job_email_classification_model")
job_stage_classification_model = SetFitModel.from_pretrained("./../models/job_stage_classification_model")
extractor = JobTitleCompanyNameExtractor()
def infer_one_sentence(model, sentence):
        predtext = [sentence]
        predicted_class = model(predtext)
        return str(predicted_class.numpy()[0])

In [28]:
def get_email_csv():
    gmail = Gmail(username="akshit.karanam99@gmail.com",password="uknf iafz dpna qyjb")
    gmail.authenticate()
    specified_date = datetime(2023, 11, 30)
    formatted_date = specified_date.strftime("%d-%b-%Y")
    ids = gmail.get_email_by_date(from_date=formatted_date)
    email_dict = gmail.parse_emails(ids[:5])
    emails_final_list = []
    for email in email_dict:
        try:
            subject = (email['subject']).decode('utf-8')
        except:
            subject = email['subject']

        html = str(BeautifulSoup(email['body']).text)
        string_list = [s.strip() for s in str(html).split()]
        final_string = ' '.join(string_list)
        final_string = "Subject: " + str(subject) +". Body: " + final_string
        stage = ""
        predicted_class = infer_one_sentence(email_clasification_model,final_string)
        if predicted_class == "1":
            stage = infer_one_sentence(model=job_stage_classification_model,sentence=final_string)
            company = extractor.get_company(final_string)
            title = extractor.get_jobtitle(final_string)
            print(stage)
            emails_final_list.append({'id':email['id'] , "date": email['date'],'Email Content': final_string,"Stage":stage})
    df = pd.DataFrame(emails_final_list)        

    file_path = './../../data/applied_data/jobs_applied.csv'

    if os.path.exists(file_path):

        df.to_csv(file_path, mode='a', index=False, header=False)
    else:
        if df.empty:
            df = pd.DataFrame(columns=['ID','Date','Email Content','Stage'])
        df.to_csv(file_path, index=False)    

In [29]:
get_email_csv()

100%|██████████| 5/5 [00:01<00:00,  2.78it/s]


NOT JOB EMAIL
NOT JOB EMAIL
NOT JOB EMAIL
NOT JOB EMAIL
NOT JOB EMAIL
